## Lecture 17 EEP118

In [ ]:
#Lecture17.R
#LECTURE 17


install.packages("pacman")
# Load the 'pacman' package
library(pacman) 
#packages to use load them now using the pacman "manager"
p_load(dplyr, haven, readr) 
#Another great feature of p_load(): if you try to load a package that is not installed on your machine, p_load() install the package for you, rather than throwing an error. For instance, let’s install and load one final package named ggplot2. 
p_load(ggplot2)



pacman::p_load(lfe, lmtest, haven, sandwich, tidyverse,car)
# lfe for running fixed effects regression
# lmtest for displaying robust SE in output table
# haven for loading in dta files
# sandwich for producing robust Var-Cov matrix
# tidyverse for manipulating data and producing plots
#car for linear hypothesis F tests


In [ ]:

#set scientific display off, thank you Roy
options(scipen=999)

#read in a Stata dataset
mydata <- read_dta("Lecture17.dta")
head(mydata)

In [ ]:
#generate category education variable
mydata$cateduc=1
mydata$cateduc[mydata$educ==12]<-2
mydata$cateduc[mydata$educ>12]<-3

#summary stats variables
summary(mydata)

### *1. Does the wage marginal effect of education differ by experience?

In [ ]:
#** LECTURE 17 STARTS HERE
##EDUC EXPER
mydata$expedu=mydata$exper*mydata$educ

six<-lm(wage~female+educ+exper+expedu, mydata)
summary(six)



Step 1: H0: beta_expEduc= 𝛽4  =0   (of the interaction term)
	    H1: 𝛽4  not equal to 0
        
Step 2  𝒕=(𝟎.𝟎𝟎𝟒−𝟎)/(𝟎.𝟎𝟑𝟑)=1.427 = 1.43

Step 3  significance level 1%  526-5 degrees of freedom c=2.58

Step 4 t< c so CANNOT reject null. 

Step 5  We cannot reject at the 1% significance level that marginal effect of experience does not depend on education.

### *2 Ordinal Variables

In [ ]:
##5. ordinal variables

mydata$HSDrop<-0
mydata$HSDrop[mydata$cateduc==1]<-1

mydata$HS<-0
mydata$HS[mydata$cateduc==2]<-1

mydata$Col<-0
mydata$Col[mydata$cateduc==3]<-1


#which specification would you choose?

seven1<-lm(wage~female+HSDrop+HS+Col+exper,mydata)
summary(seven1)

seven<-lm(wage~female+HS+Col+exper,mydata)
summary(seven)

eight<-lm(wage~female+cateduc+exper,mydata)
summary(eight)


Use adjusted R square to choose between seven1 , seven, or eight

I would choose the eight because adjusted R squared is 0.2557 greater than the other adjusted R squared


### *3. ARE the wage equations the same for males and females?

In [ ]:
###6. Are the wage equations for male and female the same?


#generate the interactions for the unrestricted model
mydata$femexper<-mydata$female*mydata$exper
mydata$femeduc<-mydata$female*mydata$educ


#do an F test

#unrestricted model 
#𝑤𝑎𝑔𝑒=𝛽_0+𝛽_1  𝑓𝑒𝑚𝑎𝑙𝑒+𝛽_2  𝑒𝑥𝑝𝑒𝑟+ 𝛽_3 𝑒𝑑𝑢+ 𝛽_4  𝑓𝑒𝑚𝑎𝑙𝑒∗𝑒𝑥𝑝𝑒𝑟+ 𝛽_5  𝑓𝑒𝑚𝑎𝑙𝑒∗𝑒𝑑𝑢 +𝑣

twelve<-lm(wage~educ+exper+female+femeduc+femexper, mydata)
summary(twelve)
#get SSR unrestricted

#restricted model 
#𝑤𝑎𝑔𝑒=𝛽_0+𝛽_2  𝑒𝑥𝑝𝑒𝑟+ 𝛽_3 𝑒𝑑𝑢 +𝑣
nine<-lm(wage~educ+exper,mydata)
summary(nine)
#get SSR restricted

#compute the F test stat and compare to the F critical value


SSR unrestricted from above output ot regression twelve is 𝑆𝑆𝑅=𝑠^2 (𝑛−𝑘)=3.018∗3.018∗520= 4736.33

SSR restricted from above output of regression nine is 𝑆𝑆𝑅𝑟=S𝑟^2 (𝑛−𝑘_𝑟 )=3.257∗3.257∗523= 5548

𝑭=((𝑺𝑺𝑹_𝑹−𝑺𝑺𝑹_𝒖)/𝒒)/(𝑺𝑺𝑹𝒖/(𝑵−𝒌−𝟏))= ((𝟓𝟓𝟒𝟖−𝟒𝟕𝟑𝟔.𝟑𝟑)/𝟑)/((𝟒𝟕𝟑𝟔.𝟑𝟑)/(𝟓𝟐𝟔−𝟓−𝟏))=𝟐𝟗.𝟔𝟓

p value of that F is 2.2e-16 ***  the p value is basically zero, has three stars

so we reject the null of male and female having the same wage equation at the 1% significance, at 5% and at 10%. 


Alternatively you could get the critical F for 1percent significance for an F with 3 and 520 degrees of freedom

3 restrictions and N-k for the unrstricted model is 520.



In [ ]:
#In R you can do the F test above by estimating twelve and then
#testing the joint null that the  female femeduc femexper are all zero (female=femeduc=femexper=0)

twelve<-lm(wage~educ+exper+female+femeduc+femexper, mydata)
summary(twelve)

linearHypothesis(twelve, c("female=0", "femeduc=0","femexper=0"))

The F is 29.65. as before

We see that the p value of that F is 0.000000000000000 *** the p value is basically zero, has three stars

so we reject the null of male and female having the same wage equation at the 1% significance, at 5% and at 10%.

Alternatively you could get the critical F for 1percent significance for an F with 3 and 520 degrees of freedom

3 restrictions and N-k for the unrstricted model is 520.

#### The Chow Test - should the model be run together males and females or as separate models?

In [ ]:
#separate equations:
ten<-lm(wage~educ+exper, mydata[which(mydata$female==1),])
summary(ten)

eleven<-lm(wage~educ+exper, mydata[which(mydata$female==0),])
summary(eleven)

#one single equation, the restricted model
nine<-lm(wage~educ+exper,mydata)
summary(nine)

#or  Chow test (new but same idea as F test)
#Model assuming same equation
#Model with two separate equations


#the Chow test is also an F test

𝑭=( (𝑺𝑺𝑹_𝑹−(𝑺𝑺𝑹_𝒇+𝑺𝑺𝑹𝒎)) / (𝒌+𝟏))  / ((𝑺𝑺𝑹𝒇+𝑺𝑺𝑹𝒎) / (𝑵𝒇−𝒌−𝟏+𝑵𝒎−𝒌−𝟏) )

In [ ]:
#run restrictec model, get SSR_r
#one single equation, the restricted model
nine<-lm(wage~educ+exper,mydata)
summary(nine)

As before SSR of nine, SRR_R = 5548


In [ ]:
#separate equations:
ten<-lm(wage~educ+exper, mydata[which(mydata$female==1),])
summary(ten)

eleven<-lm(wage~educ+exper, mydata[which(mydata$female==0),])
summary(eleven)

#and get SSR_f  and SSR_m

From Residual standard error of female in regression ten : 2.264 on 249 degrees of freedom ---> SSR_f=2.262*2.262*249= 𝟏𝟐𝟕𝟓.𝟕𝟑

From residual stad error of eleven, for males, Residual standard error: 3.574 on 271 degrees of freedom, SSR_m=3.574*3.574*271= 3462.045

Then 4737.775 =SSRf+SSRm

Substituting into the Chow F formula:

F= ((5548.159−1275.73+3462.045)/(2+1))/((1275.73+3462.045)/(252−2−1+274−2−1))=𝟐𝟗.𝟔𝟓

Given this F > 2.6 =the critical value for 5% for an F distrib with degrees of freedom (3,520)
So we reject the null that the equation should be estimated pooled into the same equation.


# Evolution of Wage gap over time

We will now investigate the evolution of the gender wage gap over time

We will use a survey dataset for 1978, a cross section of responses for that year 1978, N=550

We will also use a survey dataset for 1985, a cross section of responses for that year 1985, N=534


When we combine the two cross sections in the analysis we are pooling two cross sections, or we are doing a pooled cross section analysis

Dataset Lecture17_CPS78_85.dta

In [ ]:
#ADDITIONAL DATASET
#***use now 1978 and 1985 CPS pooled cross sections
#read in this new data set
mydata2 <- read_dta("Lecture17_CPS78_85.DTA")
head(mydata2)


#the original variable was log wage, lets get the wage level to use in the analysis below:
mydata2$wage<-exp(mydata2$lwage)

####   Create real wage


Go to CPI calculator form the BLS:
  https://www.bls.gov/data/inflation_calculator.htm

The Consumer Price Index (CPI)  is the average price
for 200 items with base in a certain year. 
1 dollar in 1978 equals 1.69 dollars in 1985.

Or get the CPI index for 1978 and for 1985 with base 
year 1982

http://www.usinflationcalculator.com/inflation/consumer-price-index-and-annual-percent-changes-from-1913-to-2008/

CPI 1978=  CPI 1978= 65.2 and CPI for 1985= 107.6

so, 

In the code cell  that comes next, we generate a new variable rwage=wage

then replace rwage=wage*107.6/65.2 if year==78 ---- for the year 78

The above command puts everything in 1985 dollars so we can compare now and combine 1978 data with 1985 data. Note that we could have put all into 1978 dollars off course, instead…


In [ ]:
#create real wage then in 1985 dollars

mydata2$rwage85<-mydata2$wage

#for 1985 we are all set, we need to put the 1978 in real 85 dollars though, that is done below:

#replace rwage=wage*107.6/65.2 if year==78
mydata2$rwage85[mydata2$year==78]<-mydata2$wage[mydata2$year==78]*107.6/65.2
mydata2$rwage85[mydata2$year==85]<-mydata2$wage[mydata2$year==85]
#label variable rwage``average real hourly earnings in 1985 dollars''



### *5. How does the gender wage gap evolve over time?

In [ ]:
reg17one<-lm(rwage85~educ+exper+union+female+y85,mydata2)
summary(reg17one)

#THE REGRESSION ABOVE, with the output below,  DOES NOT HELP US TO LOOK AT FEMALE WAGE DIFFERENCE 
#FROM MALE OVER TIME, JUST REAL WAGES OVER TIME  WITH THE  YR85 DUMMY
#and also look at female versus male one average with the FEMALE dummy



What does the coefficient estimated for y85 mean?


After controlling for gender, education, experience, union, the estimated betahat for dummy for year 1985 is -0.88. 

what does this mean?


AVERAGE WAGES IN 85 RELATIVE TO NOT 85 (YEAR 78) ARE SIGNIF LOWER BY 88 CENTS/HOUR



In [ ]:
#SO HOW CAN WE GET AT FEMALE REAL WAGE EVOLUTION FROM 78 TO 85?
#-----  INTERACT YR85 WITH FEMALE DUMMY

mydata2$y85female=mydata2$y85*mydata2$female
reg17two<-lm(rwage85~educ+exper+union+female+y85+y85female,mydata2)
summary(reg17two)

What do you see in the output above?

In 1978 females earn less -3.08 / hour on average (yr85=0)


In 1985 females earn -3.08+0.92199= -2.158/ hour on average (yr85=1)



The conditional female effect on real wages is 
(𝝏 𝒘𝒂𝒈𝒆)/(𝝏 𝒇𝒆𝒎𝒂𝒍𝒆)  = 𝛽_4 + 𝛽_6 𝑦𝑟85


= -3.08 + 0.92199 = -2.158

Conclusion, the gender gap has shrunk, before in 1978 it was -3.08 dollars/hour and in 1985 it is now -2.158


Daily Assignm: how would you generate a new variable and test whether the marginal increase in wage due
to being in a union has changed over time also from 1978 to 1985 in terms of real wage marginal effects? 



Conclusion, the gender gap has shrunk, before in 1978 it was -3.08 (women earned less 3)  dollars/hour and in 1985 it is now -2.158 (women earn now only 2.15 less)

Looking at 1996 to 2006 I found the gender gap widening, men earned more 4.1 dollars/ hour in 1996 and in 2006 men earn more 5.4 dollars/ hour… curious about more recent data, 2019, 2016 real wage gender gap…



# RECAP lecture 16 and 17 - for international's Women Day

Lecture 16:

average wage men = 7.1  average wage women=4.6   difference=2.5  dollars different average wages

on average women earn 25% less than men

looking at the distribution, missing for women mass in the 20-40 $ an hour range

wages are lower for women even after controlling for education, experience, sector of activity, and union participation. Results are Very robust

Did not show but could show by interacting female and HS, female and Col dummies that the wage differential between men and women is at all levels of education, but largest difference is for college education level

Lecture 17: 

From 1978 to 85 real wages dropped. 

Females earn less than men in 1978 and the real wage gap decreased, in that in 1985 women earn less 2.15 dollars/ hour in real wages, whereas in 85 they earned 3 dollars less

Not shown with data but, over time, 1996-2006, there is not real change at all for wages for men (coeff on yr2006) but a decline for real wages for women of 1.33 dollars.

Interesting to see what is happening in 2016/ and 2019 even…



THE END